In [1]:
%run ../jupyter/notebook_modules.ipynb

In [2]:
import bq
import train

importing Jupyter notebook from bq.ipynb
importing Jupyter notebook from competition.ipynb
importing Jupyter notebook from train.ipynb
Copying gs://natb1-kaggle-jigsaw-unintended-bias-in-toxicity-classification/train_target_on_identity_and_emotes.csv...
- [1 files][ 77.3 MiB/ 77.3 MiB]                                                
Operation completed over 1 objects/77.3 MiB.                                     


In [3]:
TRAIN_AUTOML = "train_automl"

In [22]:
bq.query(r"""
SELECT * FROM (
    SELECT
        REPLACE(comment_text, '"', r'\"') AS comment_text_escaped, 
        "toxic" AS target_class
    FROM (
        SELECT * FROM {dataset_id}.train ORDER BY RAND()
    )
    WHERE target >= 0.5
    LIMIT 50000
) UNION ALL (
    SELECT
        REPLACE(comment_text, '"', r'\"') AS comment_text_escaped, 
        "benign" AS target_class
    FROM (
        SELECT * FROM {dataset_id}.train ORDER BY RAND()
    )
    WHERE target < 0.5
    LIMIT 50000
)
""", destination=TRAIN_AUTOML)

In [7]:
bq.query("""
SELECT count(*) FROM {dataset_id}.train
""")

,f0_
0,1804874


In [23]:
bq.query("""
SELECT count(*) FROM {{dataset_id}}.{TRAIN_AUTOML}
""".format(TRAIN_AUTOML=TRAIN_AUTOML))

,f0_
0,100000


In [24]:
train.export_training_data(TRAIN_AUTOML)

Exported jigsaw_unintended_bias_in_toxicity_classification.train_automl to gs://resonant-goods-241314-lcm/train_automl.csv
